<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/03_prepare_ECMWF_L8_inputs_for_TSEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Sample input data preparation for TSEB

---


This notebook demonstrates how to setup a more realistic image collection containing the inputs required for running one of the ET models in `geeet` (TSEB). These include meteorological data (in this example we use [ECMWF data](https://doi.org/10.3390/w7126653)) and satellite data (here we use [Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands)). 

In contrast to the previous notebooks, here we compute net radiation ($R_n$) from (1) shortwave ($S_{dn}$) and longwave ($L_{dn}$) downwelling radiation, (2) albedo ($\alpha$), and (3) land surface temperature ($T_r$) using the following model (see e.g. [Zhuang and Wu, 2015](https://doi.org/10.3390/w7126653)):

$$
R_n = (1-\alpha)S_{dn} + L{dn} - \epsilon \sigma T_r^4
$$

where $\epsilon$ is the emissivity of the surface (see `geeet.solar.compute_Rn` for more details). 

>n.b. In `geeet.tseb`, this is currently the only option. `geeet.ptjpl` can accept either a net radiation band directly, or the aforementioned additional inputs to compute $Rn$. 

The sample input data prepared here can be visualized directly in this notebook (requires [geemap](https://geemap.org/installation/)), or using this [code editor sample script](https://code.earthengine.google.com/?scriptPath=users%2Flopezvoliver%2Fgeeet%3Atseb_sample_inputs_vis). 

In [1]:
# Use the following line to install geeet if needed:
#!pip install git+https://github.com/kaust-halo/geeet
# Use the following line to install geemap if needed:
#!pip install geemap
import ee
#ee.Authenticate() # Uncomment if using Google Colab or first time using EE on this device. 
ee.Initialize()

## Meteorological data

Here we get data from the ECMWF ERA-5 land-hourly forecast at 0.25°, specifically:
- surface pressure, in Pa
- air temperature, in K
- wind speed, in m/s
- downward solar and thermal radiation (shortwave and longwave components), in $W/m^2$


In [2]:
date_start = ee.Date('2021-05-01')
date_end = date_start.advance(1, 'month')

## ECMWF collection 
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)

bands_to_keep = ee.List([\
    'surface_pressure', \
    'temperature_2m', \
    'u_component_of_wind_10m',\
    'v_component_of_wind_10m',\
    'surface_solar_radiation_downwards_hourly',\
    'surface_thermal_radiation_downwards_hourly'
    ])
bands_to_rename = ee.List([\
    'surface_pressure',\
    'air_temperature', \
    'u_component_of_wind_10m',\
    'v_component_of_wind_10m',\
    'surface_solar_radiation_downwards_hourly',\
    'surface_thermal_radiation_downwards_hourly'
    ])

def prepare_ECMWF_for_TSEB(img):
    # wind speed: square root of sum of squares
    u = img.select('u_component_of_wind_10m')
    v = img.select('v_component_of_wind_10m')
    wind = ((u.pow(2)).add(v.pow(2))).pow(0.5).rename('wind_speed')

    # solar and thermal radiation conversion to W/m2 and renaming bands
    Sdn = img.select('surface_solar_radiation_downwards_hourly').divide(3600.0).rename('solar_radiation')
    Ldn = img.select('surface_thermal_radiation_downwards_hourly').divide(3600.0).rename('thermal_radiation')
    return img.addBands(wind).addBands(Sdn).addBands(Ldn)

Meteo_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterDate(date_start, date_end) \
            .select(bands_to_keep, bands_to_rename)

Meteo_collection = Meteo_collection.map(prepare_ECMWF_for_TSEB)

## Satellite data (Landsat 8)

We need thermal data to run TSEB: so here we use Landsat 8, specifically the surface reflectance product. In this test we focus on a point in Al Jawf, Saudi Arabia (lon=38.25, lat=30.25) which will correspond to a single Landsat path/row (172/39). We also filter to a specific month in order to keep the example short (only two images).

In [3]:
L8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .sort('system:time_start')

lonlat=[38.25,30.25]
geom = ee.Geometry.Point(lonlat[0], lonlat[1])
L8_collection = L8_collection.filterBounds(geom)

def process_l8(img):
    """
    Scale Landsat surface reflectance data
    Select and rename the radiometric temperature (B10)
    Compute shortwave albedo using the empirical coefficients 
    from Liang et al. (2000)
    Compute NDVI 
    """
    opticalBands = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = img.select('ST_B.*').multiply(0.00341802).add(149.0)
    Tr = thermalBands.select('ST_B10').rename('radiometric_temperature')
    # albedo short, Liang 2000 for landsat 7:
    # 0.356*B1 + 0.130*B3 + 0.373*B4 + 0.085*B5 + 0.072*B7 - 0.0018
    # equivalent for landsat 8:
    # 0.356*B2 + 0.130*B4 + 0.373*B5 + 0.085*B6 + 0.072*B7 - 0.0018
    bblue = opticalBands.select('SR_B2').multiply(0.356)
    bred = opticalBands.select('SR_B4').multiply(0.130)
    bnir = opticalBands.select('SR_B5').multiply(0.373)
    bswir = opticalBands.select('SR_B6').multiply(0.085)
    bswir2 = opticalBands.select('SR_B7').multiply(0.072)
    albedo = bblue.add(bred).add(bnir).add(bswir).add(bswir2).subtract(0.0018)
    albedo = albedo.rename('albedo')
    ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return img.addBands(opticalBands, overwrite=True)\
        .addBands(thermalBands, overwrite=True)\
        .addBands(Tr).addBands(albedo).addBands(ndvi)
    
sat_data = L8_collection.map(process_l8)

## Join meteo and sat data collections

The model we will run requires a single image at a time with all the input data given as different bands. Here we select the correct data we need for the Landsat scenes we use. 

In [4]:
# Add a datetime property to the landsat data in the same format as ECMWF
def add_datetime(img):
    d = img.get('system:time_start')
    d_parsed = ee.Date(d).format("yyyyMMdd'T'HH")
    img = img.set('system:datetime', d_parsed)
    return img

sat_data = sat_data.map(add_datetime)

# Join the two collections using the datetime property. 
filterByDateTime = ee.Filter.equals(leftField='system:datetime', rightField='system:index')
joinByDateTime = ee.ImageCollection(ee.Join.inner().apply(sat_data, Meteo_collection, filterByDateTime))
def get_img(feature):
    return ee.Image.cat(feature.get('primary'), feature.get('secondary'))

et_inputs = joinByDateTime.map(get_img)

# Provide local time offset (3 for Saudi Arabia). Required for the diurnal calculations related to 
# partitioning of solar radiation, as well as for G (diurnal variations of ground heat flux).
time_offset = 3
viewing_zenith_angle = 0 # 0 for Landsat (nadir)
zU = 10 # ECMWF: wind measured at 10m
zT = 2 # ECMWF: temperature measured at 2m
def time_properties(img):
    d = ee.Date(img.date())
    doy = d.getRelative('day','year').add(1)  # from 0-based to 1-based. 
    time = d.getRelative('hour', 'day').add(time_offset)  
    img = img.set('doy', doy)
    img = img.set('time', time)
    img = img.set('viewing_zenith', viewing_zenith_angle)
    img = img.set('zU', zU)
    img = img.set('zT', zT)
    return img

et_inputs = et_inputs.map(time_properties)

## Export

Here we export the first image as an asset over a small clipped region. 

In [5]:
region = geom.buffer(10000) # 10-km radius around the point 

task = ee.batch.Export.image.toAsset(image=et_inputs.first(),\
                                     description='TSEB_inputs_export',\
                                     assetId='projects/geeet-public/assets/TSEB_sample_inputs',\
                                     region=region,\
                                     scale=30)
task.start()

## Visualization

The inputs can be visualized in this script: https://code.earthengine.google.com/?scriptPath=users%2Flopezvoliver%2Fgeeet%3Atseb_sample_inputs_vis

Alternatively, if you have [geemap](https://geemap.org) installed, you can use the next cell to visualize the data:

In [ ]:
#!pip install geemap  # uncomment to install geemap. 
import geemap
import geemap.colormaps as cm
Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=12)
ndvi_pal = cm.palettes.ndvi
pal = cm.palettes.YlOrRd
pal_viridis = cm.palettes.viridis

Map.addLayer(et_inputs, {'bands':['SR_B4', 'SR_B3', 'SR_B2'], 'min':0, 'max': 0.3}, 'L8_SR image')
Map.addLayer(et_inputs.select('air_temperature'), {'min':20+273, 'max':50+273, 'palette':pal}, 'ECMWF Air temperature at 2m (K)', False)
Map.addLayer(et_inputs.select('surface_pressure'), {'min':9000, 'max':11000, 'palette':pal}, 'ECMWF Surface pressure (Pa)', False)
Map.addLayer(et_inputs.select('wind_speed'), {'min':0, 'max':15, 'palette':pal}, 'ECMWF wind speed (m/s)', True)
Map.addLayer(et_inputs.select('albedo'), {'min':0, 'max':1, 'palette':pal_viridis}, 'Albedo (shortwave; Liang 2000)', False)
Map.addLayer(et_inputs.select('radiometric_temperature'), {'min':20+273, 'max':50+273, 'palette':pal}, 'Landsat radiometric temperature (K)')
Map.addLayer(et_inputs.select('NDVI'), {'min':0, 'max':1, 'palette':ndvi_pal, 'opacity': 0.8}, 'NDVI')
Map

Map(center=[30.25, 38.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…